# Encodings classification 
Here we train simple classifier on encodings obtained via Wavenet autoencoder to recognize speakers

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [1]:
import numpy as np
import tensorflow as tf
from scipy.io import wavfile
from glob import glob

/home/julia/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/julia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_encodings(path):
    data, labels = [], []

    for file in glob("{}/*.npy".format(path)):
        label = file.split('/')[-1].split('-')[0]
        sample = np.load(file)

        data.append(sample[np.newaxis,:,:])
        labels.append(int(label))
        
    return np.array(data), np.array(labels)

def convert_speakers_to_labels(labels, convertion_dict = None):
    return np.array([convertion_dict[l] for l in labels], dtype = np.int32)

In [3]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, H, W, C])
    
        # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=128,
      kernel_size= [1, 3],
      padding="same",
      activation=tf.nn.relu)
    
            # Convolutional Layer #2
    conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=128,
      kernel_size= [1, 3],
      padding="same",
      activation=tf.nn.relu)
    
                # Convolutional Layer #3
    conv3 = tf.layers.conv2d(
      inputs=conv2,
      filters=128,
      kernel_size= [1, 3],
      padding="same",
      activation=tf.nn.relu)
    
                    # Convolutional Layer #3
    conv4 = tf.layers.conv2d(
      inputs=conv3,
      filters=128,
      kernel_size= [1, 3],
      padding="same",
      activation=tf.nn.relu)
    
    flat = tf.reshape(conv4, [-1, 1 * W * 128])
    dense = tf.layers.dense(inputs=flat, units=4096, activation=tf.nn.relu)

#     # Convolutional Layer #1
#     conv1 = tf.layers.conv2d(
#       inputs=input_layer,
#       filters=32,
#       kernel_size= [1, 5],
#       padding="same",
#       activation=tf.nn.relu)

#     # Pooling Layer #1
#     pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[1, 2], strides=2)

#     # Convolutional Layer #2 and Pooling Layer #2
#     conv2 = tf.layers.conv2d(
#       inputs=pool1,
#       filters=64,
#       kernel_size= [1, 5],
#       padding="same",
#       activation=tf.nn.relu)
#     pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[1, 2], strides=2)
#     print(pool2.shape)

#     # Dense Layer
#     pool2_flat = tf.reshape(pool2, [-1, 1 * 19 * 64])
#     dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=NUM_CLASSES)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [4]:
train_path = '/home/julia/DeepVoice_data/LibriSpeech_encodings/train/'
val_path = '/home/julia/DeepVoice_data/LibriSpeech_encodings/val/'
test_path = '/home/julia/DeepVoice_data/LibriSpeech_encodings/test/'


train_data, train_labels = load_encodings(train_path)
eval_data, eval_labels = load_encodings(test_path)

In [5]:
print(train_data.shape, train_labels.shape)
print(eval_data.shape, eval_labels.shape)

H, W, C = train_data.shape[1:]
NUM_CLASSES = len(set(train_labels))

(18000, 1, 78, 16) (18000,)
(4680, 1, 78, 16) (4680,)


In [6]:
NUM_CLASSES

40

In [16]:
speaker_to_label = {v:k for k,v in enumerate(set(train_labels))}

train_labels = convert_speakers_to_labels(train_labels, speaker_to_label)
eval_labels = convert_speakers_to_labels(eval_labels, speaker_to_label)

In [20]:
# Create the Estimator
model_dir = "/home/julia/DeepVoice_project/encodings_classifier/"

clsf = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir=model_dir)

tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/julia/DeepVoice_project/encodings_classifier/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f603c0d4da0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

clsf.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /home/julia/DeepVoice_project/encodings_classifier/model.ckpt.
INFO:tensorflow:probabilities = [[0.01688471 0.01758206 0.00938007 ... 0.01417516 0.03091511 0.01640902]
 [0.01203297 0.05408828 0.00765484 ... 0.01415341 0.03369755 0.02480076]
 [0.01571248 0.03525371 0.01087624 ... 0.00970396 0.06888904 0.00276336]
 ...
 [0.17773561 0.00747581 0.00007827 ... 0.00001193 0.5095254  0.00007335]
 [0.01884608 0.03091452 0.01654469 ... 0.00730657 0.03204101 0.00408874]
 [0.01677448 0.04751701 0.01415975 ... 0.0158923  0.05272238 0.00673609]]
INFO:tensorflow:loss = 4.0574274, step = 1
INFO:tensorflow:probabilities = [[0.02043661 0.02060511 0.02712553 ... 0.02501205 0.03615574 0.02433878]
 [0.03197548 0.02191939 0.02934717 ... 0.03059599 0.02041087 0.03092039]
 [0.01375898 0.02132535 0.02951997 ... 0.02413001 0.02875932 0.02804292]
 ...
 [0.0144984  0.0254676  0.01923623 ... 0.00997289 0.02556578 0.01495436]

INFO:tensorflow:loss = 3.567951, step = 701 (93.510 sec)
INFO:tensorflow:probabilities = [[0.01486826 0.02648546 0.03345847 ... 0.02640484 0.02023498 0.01500046]
 [0.02911959 0.02497749 0.0224328  ... 0.02530225 0.02272226 0.0189719 ]
 [0.02398855 0.02353867 0.01921143 ... 0.02623864 0.02516138 0.02036546]
 ...
 [0.02067989 0.03578645 0.02408696 ... 0.01971703 0.01838668 0.01842279]
 [0.0255583  0.01960515 0.02171549 ... 0.03225912 0.01693996 0.02021005]
 [0.02816894 0.02649257 0.02938569 ... 0.01343827 0.02788052 0.02021397]] (46.947 sec)
INFO:tensorflow:global_step/sec: 1.06799
INFO:tensorflow:probabilities = [[0.01690621 0.01478107 0.06105847 ... 0.06249715 0.00817019 0.00744627]
 [0.03412849 0.02475675 0.03313759 ... 0.02781691 0.01671866 0.01390999]
 [0.03106522 0.02658591 0.03148503 ... 0.01934672 0.01871306 0.01512081]
 ...
 [0.03005154 0.02752848 0.0332555  ... 0.02598482 0.01329488 0.01560542]
 [0.02744525 0.00668234 0.02887032 ... 0.05335961 0.01031774 0.0022159 ]
 [0.0198827

INFO:tensorflow:global_step/sec: 1.07889
INFO:tensorflow:probabilities = [[0.03588699 0.02280136 0.02428421 ... 0.0226227  0.03086901 0.01451504]
 [0.01236192 0.02031258 0.02975614 ... 0.00595496 0.03264241 0.00890004]
 [0.01366655 0.01634182 0.01403555 ... 0.00858104 0.03524837 0.00893037]
 ...
 [0.03455123 0.01740283 0.08257198 ... 0.03274385 0.01533079 0.01513522]
 [0.00792564 0.00838646 0.00730834 ... 0.00678718 0.02849814 0.00489734]
 [0.02169133 0.0159704  0.01298547 ... 0.01119975 0.04695765 0.01115951]] (46.374 sec)
INFO:tensorflow:loss = 3.4629376, step = 1501 (92.687 sec)
INFO:tensorflow:probabilities = [[0.04795167 0.01978025 0.04187144 ... 0.03189029 0.01611188 0.03811201]
 [0.02344877 0.01036309 0.03284519 ... 0.02957386 0.01294622 0.00384347]
 [0.02918971 0.01652823 0.03857497 ... 0.0194148  0.02496758 0.01182333]
 ...
 [0.01583905 0.01018049 0.01215775 ... 0.00996368 0.01959323 0.01275854]
 [0.01551135 0.01967809 0.01842128 ... 0.01892376 0.03006244 0.00903155]
 [0.01208

INFO:tensorflow:probabilities = [[0.01981167 0.01303417 0.03554793 ... 0.01083059 0.01853861 0.00822689]
 [0.05140831 0.02452084 0.05737937 ... 0.20853846 0.04404867 0.00081615]
 [0.01939975 0.01415921 0.03224416 ... 0.01855646 0.03232745 0.01513479]
 ...
 [0.0362     0.0219676  0.07222185 ... 0.06087952 0.01251007 0.01631042]
 [0.02231798 0.02805236 0.01683603 ... 0.00915619 0.012029   0.00936723]
 [0.01789914 0.02488833 0.00888232 ... 0.02074851 0.00968476 0.02037096]] (47.144 sec)
INFO:tensorflow:global_step/sec: 1.06441
INFO:tensorflow:probabilities = [[0.0357574  0.02233151 0.04134127 ... 0.02503841 0.02660996 0.00665794]
 [0.00520296 0.01029066 0.06633265 ... 0.00800761 0.02143226 0.0054089 ]
 [0.00176699 0.0055917  0.00330011 ... 0.00228827 0.01513507 0.00089725]
 ...
 [0.0149127  0.01188181 0.02182754 ... 0.01791538 0.01339641 0.005011  ]
 [0.08204373 0.01362202 0.02515091 ... 0.02163769 0.03015037 0.01953322]
 [0.05559342 0.02534188 0.0291194  ... 0.02207431 0.01692718 0.00933

INFO:tensorflow:loss = 2.909392, step = 3001 (95.019 sec)
INFO:tensorflow:probabilities = [[0.00758468 0.04984102 0.01963498 ... 0.00544273 0.01295251 0.01464533]
 [0.00749234 0.01008767 0.02895085 ... 0.005148   0.05088572 0.0096593 ]
 [0.02747678 0.04511909 0.01757231 ... 0.01107744 0.01266827 0.01307244]
 ...
 [0.02046476 0.03173772 0.06552833 ... 0.04097369 0.01351911 0.00577999]
 [0.01200806 0.02762412 0.03478209 ... 0.01578571 0.02254393 0.01076846]
 [0.00293068 0.00566583 0.00469635 ... 0.00670223 0.04265055 0.00204745]] (47.156 sec)
INFO:tensorflow:global_step/sec: 1.06994
INFO:tensorflow:probabilities = [[0.01472092 0.04509114 0.00714645 ... 0.01422815 0.0483887  0.06169147]
 [0.03476437 0.01937305 0.04692582 ... 0.02387048 0.04377238 0.02196108]
 [0.00341075 0.03054372 0.04483506 ... 0.00590769 0.00226128 0.02403251]
 ...
 [0.01267714 0.00580766 0.02765093 ... 0.01171796 0.04997412 0.00221263]
 [0.00591653 0.02887749 0.02554391 ... 0.01068315 0.01439103 0.02321452]
 [0.001438

...

INFO:tensorflow:global_step/sec: 0.850484
INFO:tensorflow:probabilities = [[0.0000002  0.00000902 0.00000063 ... 0.         0.         0.        ]
 [0.         0.00000011 0.00000002 ... 0.00000221 0.         0.        ]
 [0.00000011 0.11677954 0.00000001 ... 0.         0.00000003 0.        ]
 ...
 [0.00000051 0.02223236 0.00000002 ... 0.00215642 0.         0.00000109]
 [0.00000103 0.00000099 0.00001743 ... 0.00000135 0.00176932 0.00004293]
 [0.         0.         0.         ... 0.         0.         0.        ]] (59.661 sec)
INFO:tensorflow:loss = 0.019879892, step = 16601 (117.579 sec)
INFO:tensorflow:probabilities = [[0.         0.00009209 0.0007975  ... 0.         0.00000005 0.        ]
 [0.000321   0.0360843  0.9473654  ... 0.00000215 0.00000002 0.00000011]
 [0.00000003 0.00000432 0.00000122 ... 0.00000028 0.00000495 0.00000541]
 ...
 [0.00029579 0.00041114 0.00002095 ... 0.00000057 0.00000042 0.00012406]
 [0.00000002 0.00000001 0.         ... 0.         0.         0.00000003]
 [0.

INFO:tensorflow:loss = 0.00971849, step = 17301 (106.956 sec)
INFO:tensorflow:Saving checkpoints for 17346 into /home/julia/DeepVoice_project/encodings_classifier/model.ckpt.
INFO:tensorflow:probabilities = [[0.00000001 0.00000059 0.         ... 0.00000608 0.         0.        ]
 [0.00000094 0.         0.00000514 ... 0.00000005 0.         0.00001583]
 [0.0000002  0.00000224 0.00000005 ... 0.00000035 0.         0.00000013]
 ...
 [0.0000018  0.94508153 0.00000005 ... 0.00327887 0.00067014 0.        ]
 [0.         0.         0.00000153 ... 0.00000078 0.00015859 0.00000001]
 [0.         0.         0.         ... 0.         0.00000323 0.        ]] (61.422 sec)
INFO:tensorflow:global_step/sec: 0.857833
INFO:tensorflow:probabilities = [[0.00000019 0.         0.00000001 ... 0.9999988  0.         0.00000001]
 [0.         0.00000003 0.         ... 0.         0.         0.00000018]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.9999604  ...

INFO:tensorflow:global_step/sec: 1.01073
INFO:tensorflow:probabilities = [[0.         0.         0.         ... 0.00000002 0.         0.        ]
 [0.00001263 0.00025581 0.00005852 ... 0.00000003 0.00000008 0.00035785]
 [0.00000059 0.00000131 0.00000002 ... 0.00000049 0.00000587 0.00000012]
 ...
 [0.00000166 0.00000001 0.         ... 0.00000178 0.         0.00000001]
 [0.         0.00000001 0.00000125 ... 0.00000139 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]] (50.750 sec)
INFO:tensorflow:loss = 0.0030122194, step = 18101 (98.939 sec)
INFO:tensorflow:probabilities = [[0.00000083 0.0000814  0.00000003 ... 0.0000003  0.0000193  0.00000045]
 [0.00000076 0.00010433 0.00032023 ... 0.00000292 0.00005564 0.0000003 ]
 [0.00000001 0.         0.00000004 ... 0.         0.00000339 0.00000003]
 ...
 [0.         0.00001081 0.00000004 ... 0.00000122 0.         0.00000035]
 [0.00000042 0.00001026 0.00002068 ... 0.0000022  0.00000004 0.00000018]
 [0. 

INFO:tensorflow:loss = 0.0044044806, step = 18801 (96.068 sec)
INFO:tensorflow:probabilities = [[0.0006974  0.00000075 0.00001828 ... 0.00036057 0.00000004 0.00001429]
 [0.         0.         0.         ... 0.         0.00004554 0.        ]
 [0.00029344 0.00007418 0.9986821  ... 0.00068252 0.00000011 0.00000044]
 ...
 [0.00000204 0.00000313 0.         ... 0.         0.00000026 0.        ]
 [0.         0.00000053 0.00000106 ... 0.         0.00000243 0.        ]
 [0.0003161  0.         0.00000039 ... 0.00000026 0.00000012 0.        ]] (55.837 sec)
INFO:tensorflow:global_step/sec: 0.962861
INFO:tensorflow:probabilities = [[0.0000384  0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.00000002 ... 0.00000002 0.         0.        ]
 [0.9941744  0.0000172  0.0000001  ... 0.0000589  0.00000171 0.0000045 ]
 ...
 [0.00008494 0.         0.00000001 ... 0.00001015 0.00000006 0.00000308]
 [0.         0.00000061 0.         ... 0.         0.         0.        ]
 [0.

KeyboardInterrupt: 

In [22]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = clsf.evaluate(input_fn=eval_input_fn)
print(eval_results)


INFO:tensorflow:Starting evaluation at 2018-05-10-18:56:15
INFO:tensorflow:Restoring parameters from /home/julia/DeepVoice_project/encodings_classifier/model.ckpt-18532
INFO:tensorflow:Finished evaluation at 2018-05-10-18:56:29
INFO:tensorflow:Saving dict for global step 18532: accuracy = 0.17478633, global_step = 18532, loss = 5.70595
{'accuracy': 0.17478633, 'loss': 5.70595, 'global_step': 18532}
